# Wiktionary

The souce of the data was downloaded from Wikimedia dumps using the information on this page https://en.wiktionary.org/wiki/Wiktionary:Parsing. The data were accessed from https://dumps.wikimedia.org/enwiktionary/, for this particular run it was the October 21 2022 dump. One needs to download a compressed file enwiktionary-20221020-pages-articles.xml.bz2 and extract it.
Next the document is processed line by line in the first part of the following script so that at any time the contents of a single Wiktionary page are in memory.

In [1]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup as soup
import lxml
import re

# Wiktionary dump
file_name = '../wiktionary/enwiktionary-20221020-pages-articles.xml'

## Extract from XML

### Rough estimate N of Bak sentences

In [88]:
i = 0

with open(file_name, 'r') as f:
    for line in f:
        if '==Bashkir==' in line:
            i = i+1
print(i)

i = 0
with open(file_name, 'r') as f:
    for line in f:
        if 'Bashkir' in line:
            print(line)
            i = i+1
        if i > 20:
            break

2872
* Bashkir: {{t|ba|һүҙлек}}

* Bashkir: {{t|ba|ирекле|sc=Cyrl}}, {{t|ba|азат|sc=Cyrl}}, {{t|ba|бәйһеҙ|sc=Cyrl}}, {{t|ba|хөр|sc=Cyrl}}

* Bashkir: {{t|ba|иркен|sc=Cyrl}}

* Bashkir: {{t|ba|буш|sc=Cyrl}}

* Bashkir: {{t|ba|буш|sc=Cyrl}}, {{t|ba|ирекле|sc=Cyrl}}

* Bashkir: {{t|ba|түләүһеҙ|sc=Cyrl}}

* Bashkir: {{t|ba|-һыҙ|sc=Cyrl}} / {{t|ba|-һеҙ|sc=Cyrl}}, {{t|ba|-һоҙ|sc=Cyrl}} / {{t|ba|-һөҙ|sc=Cyrl}}

* Bashkir: {{tt|ba|һүҙ}}

* Bashkir: {{t|ba|быуа}}

* Bashkir: {{t|ba|бәлеш|sc=Cyrl}}

* Bashkir: {{t|ba|ҡарға|sc=Cyrl}}

* Bashkir: {{t|ba|ҡоҙғон}}

* Bashkir: {{t|ba|фил}}

* Bashkir: {{t|ba|көрән|sc=Cyrl}}

* Bashkir: {{t|ba|көрән|sc=Cyrl}}

* Bashkir: {{t|ba|ай}}

* Bashkir: {{t|ba|ғинуар|sc=Cyrl}}

* Bashkir: {{t|ba|февраль|sc=Cyrl}}

* Bashkir: {{t|ba|дүшәмбе}}

* Bashkir: {{t|ba|шишәмбе}}

* Bashkir: {{t|ba|шаршамбы}}



### Extract Bashkir sentences

In [18]:
i = 0
page_n = 0
page = False
end_page = False
document = ""
# bak_pages = []
soup_i = soup("")
examples = []

out_file_name = 'sentences.txt'

with open(file_name, 'r') as f:
    for line in f:
        if '<page>' in line:
            page_n = page_n + 1
            page = True
    
        if page:
            document = document + line
            
        if '</page>' in line:
            end_page = True
            
            soup_i = soup(document, 'lxml')
            if '==Bashkir==' in soup_i.text:
                
                if len(soup_i.title.text) > 1:
                    
                    for match in re.finditer("{{ux\|ba\|.+}}", soup_i.text):
                        with open(out_file_name, 'a') as out:
                            out.write(soup_i.text[match.start():match.end()])
                            # print(soup_i.text[match.start():match.end()])

            document = ""

## Split sentences

In [2]:
with open('sentences.txt') as g:
    text = g.readlines()

pairs = text[0].split("}}{{")

text_df = pd.DataFrame(pairs, columns = ["text"])

print(text_df.size)
print(len(pairs))

3015
3015


In [5]:
text_df.sample(n = 20, replace = False, random_state = 123456).text.tolist()

["ux|ba|Билдәле булыуынса, ташкүмер дәүерендә '''Ерҙә''' һөтимәрҙәр ҙә, ҡоштар ҙа булмаған; һаҙамыҡтарҙа ҡаҙаяҡ һәм ҡырҡбыуын ағастары үҫкән.|translation=As is known, there were neither mammals nor birds '''on the Earth''' in the Carboniferous period; ferns and equisetum grew in marshy areas.",
 "ux|ba|Һәр кеше солоғона үҙенең '''тамғаһын''' һалған.|translation=Every person put '''his/her mark''' of ownership on his/her log hive.",
 "ux|ba|'''Тимер''' рудаһы.|translation='''Iron''' ore.",
 "ux|ba|Ниңәлер '''беҙҙә''' шәреҡ ҡатын-ҡыҙҙарына йәлләү менән ҡарайҙар, йәнәhе, улар азатлығы сикле, ситлектәге ҡош хәлендә көн күрәләр.|translation=For some reason, ''(they)'' '''in our communities''' look at eastern women with pity; purportedly, they live with limited freedom, like a bird in a cage.",
 "ux|ba|Төнгә '''ҡаршы'''.|translation=When it's nearly dark, when the night is about to begin.",
 "ux|ba|'''Ҡыярҙы''' апрель аҙағында сәсәм, май аҙағында үҫентеләрҙе түтәлгә күсерәм.|translation=I pl

### Where translations can be seen

In [3]:
visible_translations = text_df.text.str.contains('\|translation=|\|t=')
text_df[visible_translations].size

2838

In [4]:
text_df[~visible_translations].sample(n = 20, random_state=1111, replace = False)

,text
2581,"ux|ba|Яман шеш йоҡмай, тибеҙ, ләкин уны булдыр..."
2503,ux|ba|'''Иртәнсәктән''' борғо һәм барабан тауы...
792,ux|ba|'''Иҙән''' таҡтаһы.|'''Floor'''board.
938,ux|ba|Бөйөк Ватан '''һуғышы'''.|inline=1|The [...
1196,ux|ba|[[Ринат]] '''Себерҙә''' [[вахта]] [[ысул...
2790,"ux|ba|Кем '''белә''', бәлки, башҡа сәбәбе лә б..."
46,ux|ba|Халыҡ һаны буйынса Франция Европала '''Г...
679,ux|ba|Тап шундай берәҙәк эттәр ваҡ мал һәм ҡош...
1006,ux|ba|Кәнфит '''ҡумтаһы'''.|A '''box''' of can...
816,ux|ba|Санкт-Петербургтағы террор '''ғәмәле'''....


In [32]:
text_split = text_df[visible_translations].text.str.replace("ux\|ba\|", "").str.split("\|t=|\|translation=", expand = True)
text_split.columns = ['bak', 'en']
text_split[1:5]

,bak,en
1,Баҙарҙан ике тоҡ '''он''' һатып алдым.,I bought two sacks of '''flour''' at the marke...
2,"Табаҡҡа биш йомортҡа һытығыҙ, '''ондо''' ҡушып...","Crack five eggs into the bowl, add '''flour'''..."
3,Ҡул '''арбаһы'''.,Hand '''cart'''.
4,"Көҙөн Сания бар донъяһын '''арбаға''' тейәп, к...","In the autumn, Saniya loaded all her belonging..."


In [29]:
text_split[text_split.isna().any(1)]

,bak,en


In [37]:
corpus_clean = pd.DataFrame()
corpus_clean['bak'] = text_split.bak.str.replace(pat = "{+|'+|}+", repl = "")
corpus_clean['en'] = text_split.en.str.replace(pat = "{+|'+|}+", repl = "")
corpus_clean[1:5]

,bak,en
1,Баҙарҙан ике тоҡ он һатып алдым.,I bought two sacks of flour at the marketplace.
2,"Табаҡҡа биш йомортҡа һытығыҙ, ондо ҡушып болға...","Crack five eggs into the bowl, add flour and s..."
3,Ҡул арбаһы.,Hand cart.
4,"Көҙөн Сания бар донъяһын арбаға тейәп, күрше а...","In the autumn, Saniya loaded all her belonging..."


In [41]:
corpus_clean.to_csv("../parallel_corpus/wiktionary_202210.csv")